### 위경도 좌표를 통해 행정구역 칼럼 추가 코드

##### 원본 데이터 스키마

|id|start time|end time|start lat|start lng|end lat|end lng|?|?|?|
|---|---|---|---|---|---|---|---|---|---|

##### 필요 라이브러리 설치


In [ ]:
#%pip install GDAL
#%pip install pyproj
%pip install Fiona
%pip install Shapely
%pip install geopandas

In [2]:
import os
import geopandas as gpd
import pandas as pd

In [ ]:
# 디렉토리 확인
#os.chdir('../')
os.getcwd()

'c:\\Users\\11\\Desktop\\LCG\\Programming\\project\\wedrive'

In [6]:
# 테스트 데이터
test_data_path=os.getcwd()+'/data/od_uuid/2024/4c17f6c58d754094ba612cef51945447.csv'
location_label_path=os.getcwd()+'/data/행정구역구분/BND_SIGUNGU_PG.shp'

In [9]:
#데이터셋 칼럼명 추가
column_name=['id','start_time','end_time','start_lat','start_lng','end_lat','end_lng','?1','?2','?3']

raw_data=pd.read_csv(test_data_path)
data=raw_data.values.tolist()
col_data=pd.DataFrame(data, columns=column_name)

#시작 위치 정보
start_loc_data=col_data[['start_lat', 'start_lng']]

#종료 위치 정보
end_loc_data=col_data[['end_lat', 'end_lng']]

In [10]:
#행정동 데이터 좌표계 변환
location_label=gpd.read_file(location_label_path, encoding='euc-kr')
location_label.to_crs(epsg=4326, inplace=True)
location_label

,BASE_DATE,SIGUNGU_NM,SIGUNGU_CD,geometry
0,20230701,종로구,11010,"POLYGON ((126.97508 37.63118, 126.97488 37.630..."
1,20230701,중구,11020,"POLYGON ((127.02314 37.57196, 127.02336 37.571..."
2,20230701,용산구,11030,"POLYGON ((126.96917 37.55488, 126.96917 37.554..."
3,20230701,성동구,11040,"POLYGON ((127.04002 37.57299, 127.04041 37.572..."
4,20230701,광진구,11050,"POLYGON ((127.10166 37.57240, 127.10224 37.572..."
...,...,...,...,...
245,20230701,함양군,38580,"POLYGON ((127.69596 35.75773, 127.69647 35.757..."
246,20230701,거창군,38590,"POLYGON ((127.88313 35.90586, 127.88299 35.905..."
247,20230701,합천군,38600,"POLYGON ((128.09861 35.83067, 128.09899 35.830..."
248,20230701,제주시,39010,"MULTIPOLYGON (((126.16791 33.28282, 126.16790 ..."


In [11]:
#시작 위치 Point 객체
start_geometry_Point=gpd.points_from_xy(start_loc_data['start_lng'], start_loc_data['start_lat'])

#종료 위치 Point 객체
end_geometry_Point=gpd.points_from_xy(end_loc_data['end_lng'], end_loc_data['end_lat'])

In [13]:
from fiona.crs import from_epsg
coulumns = ['start_lat', 'start_lng']
start_gdf_data = gpd.GeoDataFrame(start_loc_data[coulumns], geometry=start_geometry_Point, crs=from_epsg(4326))


coulumns = ['end_lat', 'end_lng']
end_gdf_data = gpd.GeoDataFrame(end_loc_data[coulumns], geometry=end_geometry_Point, crs=from_epsg(4326))

In [ ]:
from shapely.geometry import Point

start_gdf_data['start_geometry'] = start_gdf_data.apply(lambda row : Point([row['start_lng'], row['start_lat']]), axis=1)

end_gdf_data['end_geometry'] = end_gdf_data.apply(lambda row : Point([row['end_lng'], row['end_lat']]), axis=1)

0       POINT (127.13896 37.59343)
1       POINT (127.13057 37.58767)
2       POINT (127.10724 37.54673)
3       POINT (127.09419 37.54016)
4       POINT (127.08259 37.53144)
                   ...            
6952    POINT (127.07872 37.53958)
6953    POINT (127.07759 37.53932)
6954    POINT (127.08351 37.52941)
6955    POINT (127.08188 37.53674)
6956    POINT (127.08809 37.54752)
Name: end_geometry, Length: 6957, dtype: geometry

In [16]:
# start 위치의 행정구역을 추가한 데이터 프레임 생성.
df1 = gpd.sjoin(start_gdf_data, location_label)
df1.drop(columns=['index_right'], axis=1, inplace=True)

df2 = gpd.sjoin(end_gdf_data, location_label)
df2.drop(columns=['index_right'], axis=1, inplace=True)

#### 실제 위치랑 비교

In [17]:
import folium

#시작 지점 맵 띄우기
m = folium.Map(location=[df1.start_lat[0],df1.start_lng[0]],
               zoom_start=7)

path_points = start_loc_data[['start_lat', 'start_lng']].values.tolist()
for idx, p in enumerate(path_points):
    folium.Circle(p, color='blue',radius = 20, popup=idx).add_to(m)

path_points = end_loc_data[['end_lat', 'end_lng']].values.tolist()
for idx, p in enumerate(path_points):
    folium.Circle(p, color='end',radius = 20, popup=idx).add_to(m)

m.save('location_marker.html')

#### 새로운 csv 파일생성

|id|start_time|end_time|start_lat|start_lng|end_lat|end_lng|?|?|?|start_loc|end_loc|
|---|---|---|---|---|---|---|---|---|---|---|---|

In [20]:
# 새로운 csv 파일 생성
col_data['start_loc']=df1['SIGUNGU_NM']
col_data['end_loc']=df2['SIGUNGU_NM']

#경로 + /data/od_uuid_census/2024/4c17f6c58d754094ba612cef51945447.csv
new_data_path=os.getcwd()+'/data/od_uuid_census/2024/4c17f6c58d754094ba612cef51945447.csv'
col_data.to_csv(new_data_path)
col_data

,id,start_time,end_time,start_lat,start_lng,end_lat,end_lng,?1,?2,?3,start_loc,end_loc
0,4c17f6c58d754094ba612cef51945447,2024-01-01 00:33:23,2024-01-01 00:45:58,37.624019,127.140193,37.593429,127.138957,3828,4,S,구리시,구리시
1,4c17f6c58d754094ba612cef51945447,2024-01-01 10:29:24,2024-01-01 10:35:15,37.590619,127.136951,37.587667,127.130568,1126,1,S,구리시,구리시
2,4c17f6c58d754094ba612cef51945447,2024-01-01 10:35:31,2024-01-01 10:47:12,37.587170,127.130025,37.546728,127.107241,5276,6,S,구리시,광진구
3,4c17f6c58d754094ba612cef51945447,2024-01-01 10:50:41,2024-01-01 11:08:41,37.546898,127.106242,37.540164,127.094192,2863,2,S,광진구,광진구
4,4c17f6c58d754094ba612cef51945447,2024-01-01 11:09:27,2024-01-01 11:18:55,37.537902,127.093627,37.531442,127.082594,1860,1,S,광진구,광진구
...,...,...,...,...,...,...,...,...,...,...,...,...
6952,4c17f6c58d754094ba612cef51945447,2024-06-30 22:11:09,2024-06-30 22:17:53,37.535868,127.079745,37.539578,127.078724,1344,1,A,광진구,광진구
6953,4c17f6c58d754094ba612cef51945447,2024-06-30 22:18:06,2024-06-30 23:03:11,37.539735,127.078547,37.539323,127.077595,8635,8,B,광진구,광진구
6954,4c17f6c58d754094ba612cef51945447,2024-06-30 23:08:05,2024-06-30 23:21:22,37.541160,127.070320,37.529408,127.083508,2664,2,A,광진구,광진구
6955,4c17f6c58d754094ba612cef51945447,2024-06-30 23:23:58,2024-06-30 23:37:39,37.529334,127.083380,37.536735,127.081875,1885,1,A,광진구,광진구
